In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('Jobwork.csv')
df.head()

,ID,WorkPath,Statement
0,1,Machine learning,Saya dapat melakukan klasifikasi menggunakan s...
1,2,Data Analis,Saya menganalisis kumpulan data yang besar unt...
2,3,Apoteker,Saya memastikan pemberian obat yang aman dan a...
3,4,Bartender,Saya membuat koktail yang unik dan lezat yang ...
4,5,Arsitek,Saya merancang ruang yang inovatif dan fungsio...


In [4]:
statement_variable = df['Statement']
workpath_variable = df['WorkPath']

In [5]:
#Tokenize the data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(statement_variable)
max_sequence_length = max([len(tokenizer.texts_to_sequences(data)) for data in statement_variable])


In [6]:
workpath_mapping = {workpath: label for label, workpath in enumerate(set(workpath_variable)) }
train_labels = [workpath_mapping[workpath] for workpath in workpath_variable]
num_classes = len(set(train_labels))
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=num_classes)


In [7]:
# Function to preprocess and tokenize the user input
def preprocess_input(texts):
    # Convert the user input to lowercase
    texts = [text.lower() for text in texts]
    # Tokenize the user input
    sequences = tokenizer.texts_to_sequences(texts)
    # Pad the sequences to have the same length
    padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)
    return padded_sequences


In [8]:
# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=16, input_length=max_sequence_length),
    tf.keras.layers.Conv1D(32, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Create a validation set
validation_size = int(len(statement_variable) * 0.2)  # 20% of the data for validation
x_val = pad_sequences(tokenizer.texts_to_sequences(statement_variable[-validation_size:]), maxlen=max_sequence_length)
y_val = train_labels[-validation_size:]

# Train the model
model.fit(pad_sequences(tokenizer.texts_to_sequences(statement_variable), 
                        maxlen=max_sequence_length), 
                        train_labels, 
                        epochs=100, 
                        validation_data=(x_val, y_val))



Epoch 1/100
10/10 [==============================] - 2s 78ms/step - loss: 5.6553 - accuracy: 0.0000e+00 - val_loss: 5.6490 - val_accuracy: 0.0000e+00
Epoch 2/100
10/10 [==============================] - 0s 22ms/step - loss: 5.6521 - accuracy: 0.0125 - val_loss: 5.6483 - val_accuracy: 0.0156
Epoch 3/100
10/10 [==============================] - 0s 22ms/step - loss: 5.6506 - accuracy: 0.0094 - val_loss: 5.6462 - val_accuracy: 0.0156
Epoch 4/100
10/10 [==============================] - 0s 22ms/step - loss: 5.6490 - accuracy: 0.0094 - val_loss: 5.6433 - val_accuracy: 0.0156
Epoch 5/100
10/10 [==============================] - 0s 23ms/step - loss: 5.6466 - accuracy: 0.0094 - val_loss: 5.6374 - val_accuracy: 0.0156
Epoch 6/100
10/10 [==============================] - 0s 22ms/step - loss: 5.6425 - accuracy: 0.0125 - val_loss: 5.6289 - val_accuracy: 0.0156
Epoch 7/100
10/10 [==============================] - 0s 22ms/step - loss: 5.6344 - accuracy: 0.0094 - val_loss: 5.6036 - val_accuracy: 0.015

In [9]:
def predict_job(statement):
    # Preprocess input
    preprocessed_statement = preprocess_input([statement])
    
    # Make prediction
    prediction = model.predict(preprocessed_statement)[0]
    
    # Get predicted job label
    predicted_label = np.argmax(prediction)
    
    # Get job mapping
    job_mapping = {label: job for job, label in workpath_mapping.items()}
    
    # Get predicted job
    predicted_job = job_mapping[predicted_label]
    
    return predicted_job


In [12]:
#Example of predict job
statement1 = " Saya dapat melakukan klasifikasi menggunakan support vector machines "
statement2 =  " Saya menyelidiki kejahatan, mengumpulkan bukti, dan membangun kasus terhadap tersangka"
statement3 = "  Saya menganalisis sifat tanah dan batuan untuk memberikan rekomendasi untuk proyek-proyek teknik."
statement4 = " Saya mempelajari perilaku, populasi, dan habitat hewan untuk berkontribusi pada pemahaman kita tentang satwa liar."
predicted_job1 = predict_job(statement1)
predicted_job2 = predict_job(statement2)
predicted_job3 = predict_job(statement3)
predicted_job4 = predict_job(statement4)

print(predicted_job1)
print(predicted_job2)
print(predicted_job3)
print(predicted_job4)

1/1 [==============================] - 0s 22ms/step
Machine learning
Penyelidik Kriminal
Insinyur Geoteknik
Ahli Zoologi
